# Proyecto Final: Sistema de Detección de Fraude End-to-End

## 🎯 Objetivo

Crear un **sistema completo de detección de fraude** que integre:
- ✅ **Spark** - Procesamiento distribuido
- ✅ **MLflow** - Tracking y gestión de modelos
- ✅ **Airflow** - Orquestación de pipelines
- ✅ **Scikit-learn** - Modelos de ML
- ✅ **SciPy** - Análisis estadístico
- ✅ **PostgreSQL** - Almacenamiento de datos
- ✅ **Parquet** - Formato de datos eficiente
- ✅ **Matplotlib/Seaborn** - Visualizaciones
- ✅ **MLOps** - Best practices de producción

## 📋 Arquitectura del Sistema

```
┌─────────────────────────────────────────────────────────────┐
│                    PIPELINE DE DATOS                        │
├─────────────────────────────────────────────────────────────┤
│ 1. Ingesta (Spark)      → PostgreSQL                       │
│ 2. Transformación       → Parquet Files                    │
│ 3. Feature Engineering  → SciPy Statistics                 │
│ 4. Entrenamiento        → Scikit-learn + MLflow            │
│ 5. Evaluación           → Matplotlib/Seaborn               │
│ 6. Deployment          → MLflow Model Registry             │
│ 7. Monitoring          → PostgreSQL + Dashboards           │
└─────────────────────────────────────────────────────────────┘
                         Orquestado por Airflow
```

## 🚀 Escenario

**Banco XYZ** necesita un sistema automatizado para:
- Detectar transacciones fraudulentas en tiempo casi real
- Procesar millones de transacciones diarias
- Mejorar continuamente el modelo
- Cumplir con regulaciones (trazabilidad)

## 📊 Dataset

Transacciones bancarias con:
- `transaction_id`: ID único
- `customer_id`: ID del cliente
- `amount`: Monto de la transacción
- `merchant_category`: Categoría del comercio
- `transaction_hour`: Hora del día
- `is_international`: Transacción internacional (0/1)
- `card_present`: Tarjeta presente físicamente (0/1)
- `distance_from_home`: Distancia del domicilio (km)
- `is_fraud`: Target variable (0/1)

---

# PASO 1: Setup e Inicialización

Configurar todas las herramientas y conexiones

In [ ]:
# Importaciones
import os
import warnings
warnings.filterwarnings('ignore')

# Data Processing
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, mean, stddev, count, lit, udf
from pyspark.sql.types import FloatType, IntegerType, StringType
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    roc_curve, precision_recall_curve
)

# SciPy
from scipy import stats
from scipy.spatial.distance import cdist

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# MLflow
import mlflow
import mlflow.sklearn
import mlflow.spark
from mlflow.tracking import MlflowClient

# Database
from sqlalchemy import create_engine
import psycopg2

print('✅ Importaciones completadas')
print(f'📅 Fecha: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

In [ ]:
# Configuración global
PROJECT_NAME = 'fraud-detection-system'
VERSION = 'v1.0'
DATA_PATH = '../data'
LOGS_PATH = '../logs'

# Crear directorios
os.makedirs(f'{DATA_PATH}/raw', exist_ok=True)
os.makedirs(f'{DATA_PATH}/processed', exist_ok=True)
os.makedirs(f'{DATA_PATH}/parquet', exist_ok=True)
os.makedirs(LOGS_PATH, exist_ok=True)
os.makedirs('visualizations', exist_ok=True)
os.makedirs('models', exist_ok=True)

print(f'✅ Proyecto: {PROJECT_NAME} {VERSION}')
print(f'✅ Directorios creados')

In [ ]:
# Inicializar Spark
spark = SparkSession.builder \
    .appName(f'{PROJECT_NAME}-{VERSION}') \
    .master('local[*]') \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.memory', '4g') \
    .config('spark.sql.execution.arrow.pyspark.enabled', 'true') \
    .getOrCreate()

print(f'✅ Spark {spark.version} inicializado')
print(f'   Cores disponibles: {spark.sparkContext.defaultParallelism}')

In [ ]:
# Configurar MLflow
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment(f'{PROJECT_NAME}')

client = MlflowClient()

print(f'✅ MLflow configurado')
print(f'   Tracking URI: {mlflow.get_tracking_uri()}')
print(f'   Experiment: {PROJECT_NAME}')

In [ ]:
# Configurar PostgreSQL
POSTGRES_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'spark_ml_db',
    'user': 'spark_user',
    'password': 'spark_password'
}

POSTGRES_URL = f"postgresql://{POSTGRES_CONFIG['user']}:{POSTGRES_CONFIG['password']}@{POSTGRES_CONFIG['host']}:{POSTGRES_CONFIG['port']}/{POSTGRES_CONFIG['database']}"

try:
    engine = create_engine(POSTGRES_URL)
    # Test connection
    with engine.connect() as conn:
        conn.execute('SELECT 1')
    print('✅ PostgreSQL conectado')
except Exception as e:
    print(f'⚠️  PostgreSQL no disponible: {e}')
    print('   El proyecto continuará sin PostgreSQL')
    engine = None

---

# PASO 2: Generación de Datos Sintéticos

Crear dataset realista de transacciones bancarias

In [ ]:
def generate_transaction_data(n_samples=100000, fraud_ratio=0.05, seed=42):
    """
    Generar dataset sintético de transacciones bancarias
    
    Args:
        n_samples: Número total de transacciones
        fraud_ratio: Proporción de fraudes (desbalanceado)
        seed: Semilla para reproducibilidad
    
    Returns:
        pandas DataFrame con transacciones
    """
    np.random.seed(seed)
    
    # Calcular número de fraudes
    n_fraud = int(n_samples * fraud_ratio)
    n_normal = n_samples - n_fraud
    
    # Generar transacciones normales
    normal_data = {
        'amount': np.random.exponential(100, n_normal),
        'merchant_category': np.random.choice(
            ['grocery', 'restaurant', 'gas_station', 'retail', 'online'],
            n_normal,
            p=[0.3, 0.25, 0.15, 0.2, 0.1]
        ),
        'transaction_hour': np.random.choice(range(24), n_normal, p=[
            0.01, 0.01, 0.01, 0.01, 0.01, 0.02,  # 0-5am (bajo)
            0.03, 0.05, 0.06, 0.07, 0.08, 0.09,  # 6-11am (creciente)
            0.08, 0.07, 0.06, 0.05, 0.06, 0.07,  # 12-5pm (alto)
            0.08, 0.07, 0.05, 0.03, 0.02, 0.01   # 6-11pm (decreciente)
        ]),
        'is_international': np.random.choice([0, 1], n_normal, p=[0.95, 0.05]),
        'card_present': np.random.choice([0, 1], n_normal, p=[0.3, 0.7]),
        'distance_from_home': np.abs(np.random.normal(10, 20, n_normal)),
        'customer_age': np.random.randint(18, 80, n_normal),
        'num_transactions_day': np.random.poisson(3, n_normal),
        'is_fraud': np.zeros(n_normal, dtype=int)
    }
    
    # Generar transacciones fraudulentas (patrones diferentes)
    fraud_data = {
        'amount': np.random.exponential(500, n_fraud),  # Montos más altos
        'merchant_category': np.random.choice(
            ['grocery', 'restaurant', 'gas_station', 'retail', 'online'],
            n_fraud,
            p=[0.05, 0.05, 0.1, 0.3, 0.5]  # Más online y retail
        ),
        'transaction_hour': np.random.choice(
            range(24), n_fraud,
            p=[0.08]*6 + [0.02]*12 + [0.06]*6  # Más actividad nocturna
        ),
        'is_international': np.random.choice([0, 1], n_fraud, p=[0.6, 0.4]),  # Más internacional
        'card_present': np.random.choice([0, 1], n_fraud, p=[0.8, 0.2]),  # Menos presencial
        'distance_from_home': np.abs(np.random.normal(100, 50, n_fraud)),  # Más lejos
        'customer_age': np.random.randint(18, 80, n_fraud),
        'num_transactions_day': np.random.poisson(8, n_fraud),  # Más transacciones
        'is_fraud': np.ones(n_fraud, dtype=int)
    }
    
    # Combinar
    df_normal = pd.DataFrame(normal_data)
    df_fraud = pd.DataFrame(fraud_data)
    df = pd.concat([df_normal, df_fraud], ignore_index=True)
    
    # Shuffle
    df = df.sample(frac=1, random_state=seed).reset_index(drop=True)
    
    # Agregar IDs y timestamps
    df['transaction_id'] = range(len(df))
    df['customer_id'] = np.random.randint(1000, 50000, len(df))
    
    # Timestamps realistas (últimos 30 días)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30)
    df['timestamp'] = pd.to_datetime(
        np.random.randint(
            start_date.timestamp(),
            end_date.timestamp(),
            len(df)
        ),
        unit='s'
    )
    
    # Reordenar columnas
    cols = ['transaction_id', 'customer_id', 'timestamp', 'amount', 
            'merchant_category', 'transaction_hour', 'is_international',
            'card_present', 'distance_from_home', 'customer_age',
            'num_transactions_day', 'is_fraud']
    
    return df[cols]

# Generar datos
print('⏳ Generando transacciones...')
df_transactions = generate_transaction_data(n_samples=100000, fraud_ratio=0.05)

print(f'✅ {len(df_transactions):,} transacciones generadas')
print(f'   Fraudes: {df_transactions["is_fraud"].sum():,} ({df_transactions["is_fraud"].mean()*100:.2f}%)')
print(f'   Normales: {(~df_transactions["is_fraud"].astype(bool)).sum():,}')

# Mostrar muestra
df_transactions.head(10)

---

# PASO 3: Almacenamiento en PostgreSQL

Guardar datos raw en base de datos

In [ ]:
# Guardar en PostgreSQL
if engine is not None:
    try:
        print('⏳ Guardando en PostgreSQL...')
        df_transactions.to_sql(
            'transactions_raw',
            engine,
            if_exists='replace',
            index=False,
            method='multi',
            chunksize=1000
        )
        print(f'✅ {len(df_transactions):,} registros guardados en PostgreSQL')
        print('   Tabla: transactions_raw')
        
        # Verificar
        query = "SELECT COUNT(*) as total FROM transactions_raw"
        result = pd.read_sql(query, engine)
        print(f'   Verificación: {result["total"].values[0]:,} registros')
        
    except Exception as e:
        print(f'⚠️  Error guardando en PostgreSQL: {e}')
else:
    print('⚠️  PostgreSQL no disponible - continuando sin persistencia')

---

# PASO 4: Procesamiento con Spark y Parquet

Usar Spark para procesamiento distribuido y Parquet para almacenamiento eficiente

In [ ]:
# Convertir a Spark DataFrame
print('⏳ Cargando datos en Spark...')
df_spark = spark.createDataFrame(df_transactions)

print(f'✅ Spark DataFrame creado')
print(f'   Particiones: {df_spark.rdd.getNumPartitions()}')
print(f'   Registros: {df_spark.count():,}')

# Schema
print('\n📋 Schema:')
df_spark.printSchema()

In [ ]:
# Guardar como Parquet (formato columnar eficiente)
parquet_path = f'{DATA_PATH}/parquet/transactions'

print(f'⏳ Guardando en formato Parquet...')
df_spark.write.mode('overwrite').parquet(parquet_path)

print(f'✅ Datos guardados en Parquet')
print(f'   Ruta: {parquet_path}')

# Verificar tamaño
import subprocess
size = subprocess.check_output(['du', '-sh', parquet_path]).split()[0].decode('utf-8')
print(f'   Tamaño: {size}')

In [ ]:
# Leer desde Parquet (más rápido)
print('⏳ Leyendo desde Parquet...')
df_spark = spark.read.parquet(parquet_path)

print(f'✅ Datos cargados desde Parquet')
print(f'   Registros: {df_spark.count():,}')

# Mostrar muestra
df_spark.show(5)

---

# PASO 5: Análisis Exploratorio con SciPy y Visualizaciones

Análisis estadístico y visualizaciones con MLflow tracking

In [ ]:
# Iniciar run de MLflow para EDA
with mlflow.start_run(run_name='01-exploratory-data-analysis') as run:
    
    mlflow.set_tag('stage', 'exploration')
    mlflow.set_tag('version', VERSION)
    
    # Convertir a Pandas para análisis
    df_pd = df_spark.toPandas()
    
    # === Estadísticas Descriptivas ===
    print('📊 ESTADÍSTICAS DESCRIPTIVAS\n' + '='*60)
    
    stats_desc = df_pd.describe()
    print(stats_desc)
    
    # Log a MLflow
    mlflow.log_params({
        'total_transactions': len(df_pd),
        'n_customers': df_pd['customer_id'].nunique(),
        'fraud_ratio': df_pd['is_fraud'].mean(),
        'date_range_days': (df_pd['timestamp'].max() - df_pd['timestamp'].min()).days
    })
    
    # === Tests Estadísticos con SciPy ===
    print('\n🔬 TESTS ESTADÍSTICOS (SciPy)\n' + '='*60)
    
    # Test de normalidad para amount
    sample_amount = df_pd['amount'].sample(min(5000, len(df_pd)), random_state=42)
    stat, p_value = stats.shapiro(sample_amount)
    print(f'Shapiro-Wilk test (amount):")
    print(f'  Statistic: {stat:.4f}')
    print(f'  P-value: {p_value:.4e}')
    print(f'  Normal: {"No" if p_value < 0.05 else "Sí"}')
    mlflow.log_metric('amount_normality_pvalue', p_value)
    
    # Test t para diferencia de medias (fraude vs normal)
    fraud_amounts = df_pd[df_pd['is_fraud'] == 1]['amount']
    normal_amounts = df_pd[df_pd['is_fraud'] == 0]['amount']
    t_stat, t_pvalue = stats.ttest_ind(fraud_amounts, normal_amounts)
    print(f'\nT-test (fraud vs normal amounts):')
    print(f'  T-statistic: {t_stat:.4f}')
    print(f'  P-value: {t_pvalue:.4e}')
    print(f'  Diferencia significativa: {"Sí" if t_pvalue < 0.05 else "No"}')
    mlflow.log_metric('ttest_pvalue', t_pvalue)
    
    # === Visualización 1: Distribuciones ===
    print('\n📈 Creando visualizaciones...')
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Amount distribution
    axes[0, 0].hist(df_pd[df_pd['is_fraud'] == 0]['amount'], 
                    bins=50, alpha=0.6, label='Normal', color='green')
    axes[0, 0].hist(df_pd[df_pd['is_fraud'] == 1]['amount'], 
                    bins=50, alpha=0.6, label='Fraud', color='red')
    axes[0, 0].set_xlabel('Amount ($)')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Distribution of Transaction Amounts')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Hour distribution
    hour_fraud = df_pd[df_pd['is_fraud'] == 1]['transaction_hour'].value_counts().sort_index()
    hour_normal = df_pd[df_pd['is_fraud'] == 0]['transaction_hour'].value_counts().sort_index()
    
    x = np.arange(24)
    width = 0.35
    axes[0, 1].bar(x - width/2, hour_normal.values, width, label='Normal', alpha=0.8, color='green')
    axes[0, 1].bar(x + width/2, hour_fraud.values, width, label='Fraud', alpha=0.8, color='red')
    axes[0, 1].set_xlabel('Hour of Day')
    axes[0, 1].set_ylabel('Count')
    axes[0, 1].set_title('Transactions by Hour')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # Distance distribution
    axes[1, 0].boxplot(
        [df_pd[df_pd['is_fraud'] == 0]['distance_from_home'],
         df_pd[df_pd['is_fraud'] == 1]['distance_from_home']],
        labels=['Normal', 'Fraud']
    )
    axes[1, 0].set_ylabel('Distance from Home (km)')
    axes[1, 0].set_title('Distance from Home Distribution')
    axes[1, 0].grid(True, alpha=0.3, axis='y')
    
    # Merchant category
    category_counts = pd.crosstab(df_pd['merchant_category'], df_pd['is_fraud'])
    category_counts.plot(kind='bar', stacked=False, ax=axes[1, 1], color=['green', 'red'])
    axes[1, 1].set_xlabel('Merchant Category')
    axes[1, 1].set_ylabel('Count')
    axes[1, 1].set_title('Transactions by Merchant Category')
    axes[1, 1].legend(['Normal', 'Fraud'])
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('visualizations/01_eda_distributions.png', dpi=150, bbox_inches='tight')
    mlflow.log_artifact('visualizations/01_eda_distributions.png', artifact_path='exploration')
    plt.show()
    
    print('\n✅ EDA completado y guardado en MLflow')
    print(f'   Run ID: {run.info.run_id}')
    
    eda_run_id = run.info.run_id

---

# PASO 6: Feature Engineering con Spark

Crear features adicionales para mejorar el modelo

In [ ]:
with mlflow.start_run(run_name='02-feature-engineering') as run:
    
    mlflow.set_tag('stage', 'feature_engineering')
    mlflow.set_tag('version', VERSION)
    
    print('⚙️  FEATURE ENGINEERING\n' + '='*60)
    
    # === Features derivadas ===
    df_featured = df_spark \
        .withColumn('amount_log', col('amount').log1p()) \
        .withColumn('is_night', when((col('transaction_hour') < 6) | (col('transaction_hour') > 22), 1).otherwise(0)) \
        .withColumn('is_weekend', when(col('timestamp').cast('string').substr(1, 3).isin(['Sat', 'Sun']), 1).otherwise(0)) \
        .withColumn('is_high_amount', when(col('amount') > 500, 1).otherwise(0)) \
        .withColumn('is_far', when(col('distance_from_home') > 50, 1).otherwise(0)) \
        .withColumn('risk_score', 
                   col('is_international') * 0.25 + 
                   col('is_night') * 0.20 + 
                   col('is_high_amount') * 0.30 + 
                   col('is_far') * 0.25) \
        .withColumn('transactions_per_day_log', col('num_transactions_day').log1p())
    
    print('✅ Features creadas:')
    new_features = ['amount_log', 'is_night', 'is_weekend', 'is_high_amount', 
                   'is_far', 'risk_score', 'transactions_per_day_log']
    for feat in new_features:
        print(f'   • {feat}')
    
    mlflow.log_param('new_features_count', len(new_features))
    mlflow.log_param('new_features', ', '.join(new_features))
    
    # === Encoding categórico ===
    print('\n⚙️  Encoding categórico...')
    indexer = StringIndexer(inputCol='merchant_category', outputCol='merchant_category_idx')
    df_featured = indexer.fit(df_featured).transform(df_featured)
    
    print('✅ merchant_category → merchant_category_idx')
    
    # === Mostrar muestra ===
    print('\n📋 Muestra de features:')
    df_featured.select('amount', 'amount_log', 'risk_score', 'is_fraud').show(5)
    
    # === Guardar featured data ===
    featured_parquet_path = f'{DATA_PATH}/parquet/transactions_featured'
    df_featured.write.mode('overwrite').parquet(featured_parquet_path)
    
    print(f'\n✅ Featured data guardada en: {featured_parquet_path}')
    
    feature_run_id = run.info.run_id

---

# PASO 7: Preparación de Datos para ML

Crear features vectors y split train/test

In [ ]:
# Leer featured data
df_featured = spark.read.parquet(featured_parquet_path)

# Definir features para el modelo
feature_cols = [
    'amount_log',
    'merchant_category_idx',
    'transaction_hour',
    'is_international',
    'card_present',
    'distance_from_home',
    'customer_age',
    'num_transactions_day',
    'is_night',
    'is_high_amount',
    'is_far',
    'risk_score',
    'transactions_per_day_log'
]

print(f'📊 Features seleccionadas: {len(feature_cols)}')
for i, feat in enumerate(feature_cols, 1):
    print(f'   {i:2d}. {feat}')

# Ensamblar features
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features_raw')
df_assembled = assembler.transform(df_featured)

# Escalar features
scaler = StandardScaler(inputCol='features_raw', outputCol='features', withMean=True, withStd=True)
scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)

# Seleccionar columnas finales
df_ml = df_scaled.select('features', col('is_fraud').alias('label'))

print('\n✅ Data preparada para ML')
print(f'   Features: vector de {len(feature_cols)} dimensiones')
print(f'   Registros: {df_ml.count():,}')

# Split train/test
train_spark, test_spark = df_ml.randomSplit([0.8, 0.2], seed=42)

print(f'\n📊 Train/Test Split:')
print(f'   Train: {train_spark.count():,} ({train_spark.count()/df_ml.count()*100:.1f}%)')
print(f'   Test:  {test_spark.count():,} ({test_spark.count()/df_ml.count()*100:.1f}%)')

In [ ]:
# Convertir a Pandas/Numpy para Scikit-learn
print('⏳ Convirtiendo a formato Scikit-learn...')

# Extraer features y labels
def spark_to_sklearn(df_spark):
    df_pd = df_spark.toPandas()
    X = np.array([row.toArray() for row in df_pd['features']])
    y = df_pd['label'].values
    return X, y

X_train, y_train = spark_to_sklearn(train_spark)
X_test, y_test = spark_to_sklearn(test_spark)

print(f'✅ Datos convertidos')
print(f'   X_train: {X_train.shape}')
print(f'   X_test:  {X_test.shape}')
print(f'   Fraud rate train: {y_train.mean()*100:.2f}%')
print(f'   Fraud rate test:  {y_test.mean()*100:.2f}%')

---

# PASO 8: Entrenamiento de Modelos con Scikit-learn

Entrenar y comparar múltiples modelos con MLflow tracking

In [ ]:
# Definir modelos a entrenar
models_to_train = {
    'RandomForest': RandomForestClassifier(
        n_estimators=100,
        max_depth=15,
        min_samples_split=10,
        random_state=42,
        n_jobs=-1
    ),
    'GradientBoosting': GradientBoostingClassifier(
        n_estimators=100,
        max_depth=7,
        learning_rate=0.1,
        random_state=42
    ),
    'LogisticRegression': LogisticRegression(
        max_iter=1000,
        random_state=42,
        n_jobs=-1
    )
}

print(f'🤖 Modelos a entrenar: {len(models_to_train)}')
for name in models_to_train.keys():
    print(f'   • {name}')

In [ ]:
# Entrenar todos los modelos
import time

results = []

print('\n' + '='*70)
print('ENTRENAMIENTO DE MODELOS')
print('='*70 + '\n')

for model_name, model in models_to_train.items():
    
    print(f'\n🚀 Entrenando: {model_name}')
    print('-' * 70)
    
    with mlflow.start_run(run_name=f'03-model-{model_name.lower()}') as run:
        
        mlflow.set_tag('stage', 'training')
        mlflow.set_tag('model_type', model_name)
        mlflow.set_tag('version', VERSION)
        
        # Log parámetros del modelo
        params = model.get_params()
        mlflow.log_params({k: v for k, v in params.items() if isinstance(v, (int, float, str, bool))})
        
        # Entrenar
        start_time = time.time()
        model.fit(X_train, y_train)
        train_time = time.time() - start_time
        
        print(f'  ⏱️  Tiempo de entrenamiento: {train_time:.2f}s')
        
        # Predicciones
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        y_pred_proba_test = model.predict_proba(X_test)[:, 1]
        
        # Métricas
        train_acc = accuracy_score(y_train, y_pred_train)
        test_acc = accuracy_score(y_test, y_pred_test)
        test_precision = precision_score(y_test, y_pred_test)
        test_recall = recall_score(y_test, y_pred_test)
        test_f1 = f1_score(y_test, y_pred_test)
        test_auc = roc_auc_score(y_test, y_pred_proba_test)
        
        metrics = {
            'train_accuracy': train_acc,
            'test_accuracy': test_acc,
            'test_precision': test_precision,
            'test_recall': test_recall,
            'test_f1': test_f1,
            'test_auc_roc': test_auc,
            'train_time_seconds': train_time
        }
        
        mlflow.log_metrics(metrics)
        
        # Imprimir métricas
        print(f'\n  📊 Métricas:')
        print(f'     Train Accuracy: {train_acc:.4f}')
        print(f'     Test Accuracy:  {test_acc:.4f}')
        print(f'     Precision:      {test_precision:.4f}')
        print(f'     Recall:         {test_recall:.4f}')
        print(f'     F1-Score:       {test_f1:.4f}')
        print(f'     AUC-ROC:        {test_auc:.4f}')
        
        # Guardar modelo
        mlflow.sklearn.log_model(
            model, 
            f'{model_name.lower()}-model',
            registered_model_name=f'fraud-detection-{model_name.lower()}'
        )
        
        # Guardar resultados
        results.append({
            'model': model_name,
            'run_id': run.info.run_id,
            **metrics,
            'model_object': model,
            'y_pred': y_pred_test,
            'y_pred_proba': y_pred_proba_test
        })
        
        print(f'\n  ✅ Modelo guardado en MLflow')
        print(f'     Run ID: {run.info.run_id}')

print('\n' + '='*70)
print('✅ TODOS LOS MODELOS ENTRENADOS')
print('='*70)

---

# PASO 9: Comparación y Selección del Mejor Modelo

Visualizar y comparar todos los modelos

In [ ]:
# Crear DataFrame de resultados
df_results = pd.DataFrame([{
    'Model': r['model'],
    'Accuracy': r['test_accuracy'],
    'Precision': r['test_precision'],
    'Recall': r['test_recall'],
    'F1-Score': r['test_f1'],
    'AUC-ROC': r['test_auc_roc'],
    'Train Time (s)': r['train_time_seconds']
} for r in results])

print('📊 COMPARACIÓN DE MODELOS\n' + '='*70)
print(df_results.to_string(index=False))
print('='*70)

# Mejor modelo por F1-Score (métrica balanceada para datos desbalanceados)
best_model_idx = df_results['F1-Score'].idxmax()
best_model_name = df_results.loc[best_model_idx, 'Model']
best_model_f1 = df_results.loc[best_model_idx, 'F1-Score']

print(f'\n🏆 MEJOR MODELO: {best_model_name} (F1={best_model_f1:.4f})')

In [ ]:
# Visualización de comparación
with mlflow.start_run(run_name='04-model-comparison') as run:
    
    mlflow.set_tag('stage', 'comparison')
    mlflow.set_tag('best_model', best_model_name)
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    
    for idx, metric in enumerate(metrics_to_plot):
        row, col = idx // 2, idx % 2
        
        bars = axes[row, col].bar(df_results['Model'], df_results[metric], 
                                   color=['gold' if m == best_model_name else 'steelblue' 
                                         for m in df_results['Model']])
        axes[row, col].set_ylabel(metric, fontsize=12)
        axes[row, col].set_title(f'{metric} Comparison', fontweight='bold', fontsize=14)
        axes[row, col].set_ylim([0, 1])
        axes[row, col].grid(True, alpha=0.3, axis='y')
        
        # Añadir valores en las barras
        for bar in bars:
            height = bar.get_height()
            axes[row, col].text(bar.get_x() + bar.get_width()/2., height,
                               f'{height:.3f}',
                               ha='center', va='bottom', fontsize=10)
    
    plt.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold', y=1.00)
    plt.tight_layout()
    plt.savefig('visualizations/04_model_comparison.png', dpi=150, bbox_inches='tight')
    mlflow.log_artifact('visualizations/04_model_comparison.png')
    plt.show()
    
    # Log best model info
    mlflow.log_metrics({
        f'best_{k}': v for k, v in df_results.loc[best_model_idx].items()
        if isinstance(v, (int, float))
    })
    
    comparison_run_id = run.info.run_id

print(f'\n✅ Comparación guardada en MLflow (Run ID: {comparison_run_id})')

---

# PASO 10: Evaluación Detallada del Mejor Modelo

Análisis completo con visualizaciones

In [ ]:
# Obtener mejor modelo
best_result = [r for r in results if r['model'] == best_model_name][0]
best_model = best_result['model_object']
y_pred = best_result['y_pred']
y_pred_proba = best_result['y_pred_proba']

with mlflow.start_run(run_name=f'05-best-model-evaluation-{best_model_name.lower()}') as run:
    
    mlflow.set_tag('stage', 'final_evaluation')
    mlflow.set_tag('model', best_model_name)
    mlflow.set_tag('production_ready', 'true')
    
    print('🔍 EVALUACIÓN DETALLADA\n' + '='*70)
    
    # === Classification Report ===
    print('\n📋 Classification Report:\n')
    report = classification_report(y_test, y_pred, target_names=['Normal', 'Fraud'])
    print(report)
    
    # Guardar report
    with open('models/classification_report.txt', 'w') as f:
        f.write(report)
    mlflow.log_artifact('models/classification_report.txt')
    
    # === Confusion Matrix ===
    cm = confusion_matrix(y_test, y_pred)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Absolute
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
               xticklabels=['Normal', 'Fraud'],
               yticklabels=['Normal', 'Fraud'])
    axes[0].set_title('Confusion Matrix (Counts)', fontweight='bold')
    axes[0].set_ylabel('Actual')
    axes[0].set_xlabel('Predicted')
    
    # Normalized
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Greens', ax=axes[1],
               xticklabels=['Normal', 'Fraud'],
               yticklabels=['Normal', 'Fraud'])
    axes[1].set_title('Confusion Matrix (Normalized)', fontweight='bold')
    axes[1].set_ylabel('Actual')
    axes[1].set_xlabel('Predicted')
    
    plt.tight_layout()
    plt.savefig('visualizations/05_confusion_matrix.png', dpi=150, bbox_inches='tight')
    mlflow.log_artifact('visualizations/05_confusion_matrix.png')
    plt.show()
    
    # === ROC & PR Curves ===
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_proba)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # ROC
    axes[0].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {roc_auc:.3f})')
    axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
    axes[0].set_xlabel('False Positive Rate')
    axes[0].set_ylabel('True Positive Rate')
    axes[0].set_title('ROC Curve', fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # PR
    axes[1].plot(recall_curve, precision_curve, color='green', lw=2, label='PR Curve')
    axes[1].set_xlabel('Recall')
    axes[1].set_ylabel('Precision')
    axes[1].set_title('Precision-Recall Curve', fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('visualizations/05_roc_pr_curves.png', dpi=150, bbox_inches='tight')
    mlflow.log_artifact('visualizations/05_roc_pr_curves.png')
    plt.show()
    
    # === Feature Importance ===
    if hasattr(best_model, 'feature_importances_'):
        importances = best_model.feature_importances_
        indices = np.argsort(importances)[::-1]
        
        plt.figure(figsize=(12, 8))
        importance_df = pd.DataFrame({
            'feature': [feature_cols[i] for i in indices],
            'importance': importances[indices]
        })
        
        sns.barplot(data=importance_df, y='feature', x='importance', palette='viridis')
        plt.title(f'Feature Importance - {best_model_name}', fontweight='bold', fontsize=14)
        plt.xlabel('Importance')
        plt.tight_layout()
        plt.savefig('visualizations/05_feature_importance.png', dpi=150, bbox_inches='tight')
        mlflow.log_artifact('visualizations/05_feature_importance.png')
        plt.show()
        
        # Guardar CSV
        importance_df.to_csv('models/feature_importance.csv', index=False)
        mlflow.log_artifact('models/feature_importance.csv')
    
    print('\n✅ Evaluación completa guardada en MLflow')
    print(f'   Run ID: {run.info.run_id}')
    
    evaluation_run_id = run.info.run_id

---

# PASO 11: Guardar Resultados en PostgreSQL

Persistir predicciones y métricas

In [ ]:
if engine is not None:
    
    print('💾 GUARDANDO RESULTADOS EN POSTGRESQL\n' + '='*70)
    
    # === Tabla de predicciones ===
    df_predictions = df_transactions.iloc[test_spark.toPandas().index].copy()
    df_predictions['predicted_fraud'] = y_pred
    df_predictions['fraud_probability'] = y_pred_proba
    df_predictions['model_name'] = best_model_name
    df_predictions['prediction_date'] = datetime.now()
    
    try:
        df_predictions.to_sql(
            'fraud_predictions',
            engine,
            if_exists='replace',
            index=False
        )
        print(f'✅ {len(df_predictions):,} predicciones guardadas en tabla: fraud_predictions')
    except Exception as e:
        print(f'⚠️  Error: {e}')
    
    # === Tabla de métricas de modelos ===
    df_model_metrics = df_results.copy()
    df_model_metrics['evaluation_date'] = datetime.now()
    df_model_metrics['version'] = VERSION
    
    try:
        df_model_metrics.to_sql(
            'model_metrics',
            engine,
            if_exists='append',
            index=False
        )
        print(f'✅ Métricas de {len(df_model_metrics)} modelos guardadas en tabla: model_metrics')
    except Exception as e:
        print(f'⚠️  Error: {e}')
    
    print('\n✅ Todos los resultados guardados en PostgreSQL')
    
else:
    print('⚠️  PostgreSQL no disponible - saltando persistencia')

---

# PASO 12: Resumen Final y MLOps

Resumen del proyecto y próximos pasos

In [ ]:
print('\n' + '='*70)
print('RESUMEN DEL PROYECTO')
print('='*70)

print(f'\n📊 Dataset:')
print(f'   Total transacciones: {len(df_transactions):,}')
print(f'   Fraudes: {df_transactions["is_fraud"].sum():,} ({df_transactions["is_fraud"].mean()*100:.2f}%)')
print(f'   Periodo: {(df_transactions["timestamp"].max() - df_transactions["timestamp"].min()).days} días')

print(f'\n🔧 Feature Engineering:')
print(f'   Features originales: {len(df_transactions.columns)}')
print(f'   Features creadas: {len(new_features)}')
print(f'   Features finales para ML: {len(feature_cols)}')

print(f'\n🤖 Modelos Entrenados:')
for r in results:
    print(f'   • {r["model"]:20s} - F1: {r["test_f1"]:.4f}, AUC: {r["test_auc_roc"]:.4f}')

print(f'\n🏆 Mejor Modelo: {best_model_name}')
print(f'   Accuracy:  {best_result["test_accuracy"]:.4f}')
print(f'   Precision: {best_result["test_precision"]:.4f}')
print(f'   Recall:    {best_result["test_recall"]:.4f}')
print(f'   F1-Score:  {best_result["test_f1"]:.4f}')
print(f'   AUC-ROC:   {best_result["test_auc_roc"]:.4f}')

print(f'\n💾 Artefactos Generados:')
print(f'   • Parquet files: {DATA_PATH}/parquet/')
print(f'   • Visualizaciones: visualizations/')
print(f'   • Modelos: MLflow Model Registry')
if engine:
    print(f'   • PostgreSQL: fraud_predictions, model_metrics')

print(f'\n🔗 MLflow Runs:')
print(f'   • EDA: {eda_run_id[:8]}...')
print(f'   • Feature Engineering: {feature_run_id[:8]}...')
print(f'   • Comparación: {comparison_run_id[:8]}...')
print(f'   • Mejor Modelo: {evaluation_run_id[:8]}...')

print(f'\n🌐 Servicios:')
print(f'   • MLflow UI: http://localhost:5000')
print(f'   • PostgreSQL: localhost:5432')

print('\n' + '='*70)
print('✅ PROYECTO COMPLETADO')
print('='*70)

---

# PASO 13: Próximos Pasos - MLOps

## Implementación en Producción

### 1. Automatización con Airflow

Crear DAG en `airflow/dags/fraud_detection_pipeline.py`:

```python
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta

default_args = {
    'owner': 'fraud-detection-team',
    'start_date': datetime(2024, 1, 1),
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
}

with DAG(
    'fraud_detection_pipeline',
    default_args=default_args,
    schedule_interval='@daily',
    catchup=False
) as dag:
    
    ingest = PythonOperator(
        task_id='ingest_data',
        python_callable=ingest_data_func
    )
    
    transform = PythonOperator(
        task_id='transform_data',
        python_callable=transform_data_func
    )
    
    predict = PythonOperator(
        task_id='predict_fraud',
        python_callable=predict_func
    )
    
    ingest >> transform >> predict
```

### 2. Deployment del Modelo

```python
# Cargar modelo desde MLflow
model_uri = f"models:/fraud-detection-{best_model_name.lower()}/Production"
loaded_model = mlflow.sklearn.load_model(model_uri)

# API Flask para predicciones
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    prediction = loaded_model.predict_proba([data['features']])[0][1]
    return jsonify({'fraud_probability': float(prediction)})
```

### 3. Monitoring

- Métricas de modelo en producción
- Detección de data drift
- Alertas de degradación de performance

### 4. Retraining

- Entrenar automáticamente con nuevos datos
- A/B testing de modelos
- Promoción automática a producción

## ¡Felicidades!

Has completado un proyecto end-to-end que integra:
✅ Spark, ✅ MLflow, ✅ Airflow, ✅ Scikit-learn, ✅ SciPy, 
✅ PostgreSQL, ✅ Parquet, ✅ Matplotlib/Seaborn

Este proyecto demuestra las mejores prácticas de MLOps y está listo para producción.